<a href="https://colab.research.google.com/github/howawong/7M18gX72n02RA968WyVbiA/blob/master/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install transformers kora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.3 MB/s eta 0:00:00


In [12]:
import pandas as pd


serp_result = pd.read_csv('/content/drive/MyDrive/Klook/cleaned_organic_serp.csv', na_filter=True)
serp_result = serp_result[~serp_result['snippet'].isnull()]
serp_result = serp_result.groupby('query')['snippet'].agg(lambda x: ' '.join(x)).reset_index()

In [40]:
def chunk_strings(string_list, chunk_size):
    return [string_list[i:i + chunk_size] for i in range(0, len(string_list), chunk_size)]


def predict_labels(queries, model, tokenizer, multi_class):
    inputs = tokenizer(queries, padding=True, truncation=True, return_tensors="pt")
    inputs.to('cuda')
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    probabilities = torch.sigmoid(logits).cpu()
    if multi_class:
        predicted_labels = torch.argmax(probabilities, dim=1).numpy()
    else:
        predicted_labels = (probabilities > 0.5).int().numpy()

    return predicted_labels

def labels_to_text(labels):
    def _process(l):
      return [mlb_saved.classes_[idx] for idx, x in enumerate(l) if x == 1]
    print(labels)
    print(mlb_saved.classes_)
    return [_process(l) for l in labels]

def predict_chunks(texts, model, tokenizer, multi_class):
    predictions = []
    for chunk in chunk_strings(texts, 50):
        predictions += predict_labels(chunk, model, tokenizer, multi_class).tolist()
    return predictions

In [28]:
import torch
import joblib
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay

# Specify the path to the saved model
model_path = '/content/drive/MyDrive/Klook/intent_model'
mlb_save_path = '/content/drive/MyDrive/Klook/intent_model/mlb.pkl'
mlb_saved = joblib.load(mlb_save_path)
# Load the tokenizer and model
intent_tokenizer = AutoTokenizer.from_pretrained(model_path)
intent_model = AutoModelForSequenceClassification.from_pretrained(model_path)
intent_model.to('cuda')

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [48]:
serp_result_with_intent = serp_result[['query', 'snippet']]
predictions = predict_chunks(serp_result_with_intent['snippet'].tolist(), intent_model, intent_tokenizer, False)
serp_result_with_intent['prediction'] = [set([mlb_saved.classes_[idx] for idx, x in enumerate(l) if x == 1])  for l in predictions]
for c in mlb_saved.classes_:
  serp_result_with_intent[c] = serp_result_with_intent['prediction'].apply(lambda x: 1 if c in x else 0)
serp_result_with_intent = serp_result_with_intent.drop(columns=['prediction', 'snippet'])
serp_result_with_intent

,query,Airport Or Flight,Aquarium / Zoo,Art Museum / Gallery,Beach,Bus,Car Rental,Casino,Cruise,Ferry,...,"Night Life, Bar, Club",Parking,Resort,Skiing / Snowboard,Spa / Massage / Treatment,Theme Park,Train,Villa,Waterfall Or Lake,Winery / Vineyard
0,$39 busch gardens tickets,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,$45 universal tickets,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,$50 disneyland tickets,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,$50 disneyland tickets 2023,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,$59 disney world tickets 2023,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50300,힐튼 도쿄,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50301,힐튼 도쿄 오다이바,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
50302,힐튼 오사카,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
50303,힐튼 오키나와 차탄 리조트,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [31]:
# Specify the path to the saved model
model_path = '/content/drive/MyDrive/Klook/location_model'
label_mapping_path = '/content/drive/MyDrive/Klook/location_label_mapping.csv'
label_mapping = pd.read_csv(label_mapping_path)
id2label = {r['id']: r['label'] for r in label_mapping.to_dict('records')}
# Load the tokenizer and model
location_tokenizer = AutoTokenizer.from_pretrained(model_path)
location_model = AutoModelForSequenceClassification.from_pretrained(model_path)
location_model.to('cuda')

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [49]:
serp_result_with_location = serp_result[['query', 'snippet']]
predictions = predict_chunks(serp_result_with_location['snippet'].tolist(), location_model, location_tokenizer, True)
serp_result_with_location['prediction'] = [id2label[l].split(' / ') for l in predictions]
serp_result_with_location['country'] = serp_result_with_location['prediction'].apply(lambda x: x[0])
serp_result_with_location['city'] = serp_result_with_location['prediction'].apply(lambda x: x[1] if len(x) > 1 else '')
serp_result_with_location = serp_result_with_location.drop(columns=['prediction', 'snippet'])
serp_result_with_location

,query,country,city
0,$39 busch gardens tickets,United States,
1,$45 universal tickets,United States,Florida
2,$50 disneyland tickets,Rest of the World,
3,$50 disneyland tickets 2023,United States,Florida
4,$59 disney world tickets 2023,United States,Florida
...,...,...,...
50300,힐튼 도쿄,Japan,Tokyo
50301,힐튼 도쿄 오다이바,Japan,Tokyo
50302,힐튼 오사카,Japan,Osaka
50303,힐튼 오키나와 차탄 리조트,Japan,Okinawa


In [50]:
output  = serp_result_with_location.merge(serp_result_with_intent, on=['query'], how='inner')
output.to_csv('/content/drive/MyDrive/Klook/inference.csv', index=False)